# Análise de texto de fontes desestruturadas e Web

## Aula 02 - PDFs

Nesta aula iremos trabalhar com extração de informações a partir de PDFs utilizando bibliotecas do Python.

## O que são arquivos PDF?

O PDF, ou *Portable Document Format*, é um formato de arquivo criado pela Adobe em 1993. O principal objetivo em utilizar um PDF envolve permitir que um documento seja lido de forma independente do hardware ou sistema operacional utilizado pelo usuário. Também, sem a necessidade de possuir instalado o programa utilizado para criar o documento.

<img src="https://atd-insper.s3.us-east-2.amazonaws.com/aula02/pdf_adobe.jpg">

Documentos PDF são padronizados conforme a ISO 32000. Os arquivos PDF geralmente incluem:
- informações textuais
- imagens vetoriais
- imagens bitmap

Eles são distribuídos principalmente com foco em leitura, ou seja, ao criar um DOC no Microsoft Word, repassamos o `arquivo.DOCX` para pessoas que necessitem editá-lo.

Para as pessoas que apenas necessitam ler as informações do documento, transformamos o DOC em PDF e enviamos neste formato!


## Qual a importância de extrair informações de PDFs?

Arquivos PDF são criados com foco principal sendo a interação entre organizações e pessoas (uma cria, a outra lê).

Em um cenário ideal, as informações contidas no PDF também estariam disponíveis em um ambiente estruturado e de fácil recuperação (ex: sistema de gerenciamento de banco de dados). Entretanto, muitas vezes a única forma disponível para acessos aos dados é o próprio arquivo PDF, por exemplo: editais, relatórios governamentais, lista de lojas, relatórios de desempenho, etc.

Nesta aula, iremos aprender como extrair informações de PDFs, tabulares ou não. Além disso, veremos como utilizar recursos do Python para armazenar e analisar estes dados.

## Biblioteca *camelot*
Para os dados tabulares, a principal biblioteca que utilizaremos será a camelot (https://camelot-py.readthedocs.io/en/master/).
Para utilizá-la, será necessário instalar algumas dependências. Como você está utilizando o **Google Colab**, execute os seguintes comandos (execute esta célula):

In [ ]:
!pip install ghostscript
!pip install camelot-py[cv]
!pip install excalibur-py
!pip install 'PyPDF2<3.0'
!sudo apt update
!sudo apt install ghostscript -y

Lembra-se de executar o import novamente quando instalar bibliotecas!

## Importando as bibliotecas necessárias

Aqui, iremos importar as bibliotecas que utilizaremos na aula

In [ ]:
import os
import pandas as pd
import numpy as np
import camelot
import missingno as mn
import seaborn as sb
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

In [ ]:
# !pip install plotly

Caso obtenha algum erro, utilize o **!pip install** para instalar a biblioteca ausente!

In [ ]:
# Exemplo (remover # da proxima linha se preciso)
!pip install missingno

# libGl missing (https://github.com/opencv/opencv-python/issues/370#issuecomment-671202529)
# !pip install opencv-python-headless

Vamos conferir em qual diretório estamos executando o notebook?

In [ ]:
print('O seu notebook está no diretório')
print(os.getcwd())


## Lendo um PDF

Para realizar a leitura dos dados tabulares, vamos utilizar a biblioteca **camelot**:

In [ ]:
pdf_path = 'https://atd-insper.s3.us-east-2.amazonaws.com/aula02/stores.pdf'
tabelas = camelot.read_pdf(pdf_path)
tabelas

Podemos ver que encontramos uma tabela no documento.

Agora, vamos conferir o *parsing report* do *camelot* para esta tabela:

In [ ]:
print(tabelas[0].parsing_report)

E podemos obter um *DataFrame* do *pandas* com os dados desta tabela:

Depois iremos aprender o que mais podemos fazer com este DataFrame!

In [ ]:
df = tabelas[0].df
df

A funcionalidade de *read_pdf* do *camelot* permite especificar diversos parâmetros. Por exemplo, podemos ler apenas a página 2:

In [ ]:
pdf_path = 'https://atd-insper.s3.us-east-2.amazonaws.com/aula02/stores.pdf'
tabelas = camelot.read_pdf(pdf_path, pages = '2')
df_pg2 = tabelas[0].df
df_pg2

# Múltiplas tabelas em um mesmo arquivo

Imagem de exemplo com múltiplas tabelas em uma mesma página:
<img src="https://atd-insper.s3.us-east-2.amazonaws.com/aula02/relatorio_aula02.png">

In [ ]:
pdf_path = 'https://atd-insper.s3.us-east-2.amazonaws.com/aula02/relatorio.pdf'
tab_rep = camelot.read_pdf(pdf_path)
tab_rep

Podemos perceber que foram encontradas duas tabelas no arquivo.

In [ ]:
df_relat1 = tab_rep[0].df
df_relat1

In [ ]:
df_relat2 = tab_rep[1].df
df_relat2

Utilizando o report gráfico para analisar as extrações:

In [ ]:
camelot.plot(tab_rep[1], kind='text').show()

In [ ]:
camelot.plot(tab_rep[0], kind='contour').show()

## Primeiro contato com o *pandas*

Agora, vamos aprender como utilizar recursos da biblioteca *pandas*.

In [ ]:
pdf_path = 'https://atd-insper.s3.us-east-2.amazonaws.com/aula02/stores.pdf'
tabelas = camelot.read_pdf(pdf_path, split_text=True, pages='1-end')
tabelas

In [ ]:
type(tabelas[0].df)

Iremos concatenar dois DataFrames:

In [ ]:
df = pd.concat([tabelas[0].df, tabelas[1].df])

podemos exibir apenas o início do nosso DataFrame, para não ter a tela toda poluída com dados

In [ ]:
df.head()

ou os últimos:

In [ ]:
df.tail()

é possível também especificar o número de linhas

In [ ]:
df.head(2)

Perceberam que o nome das colunas ficou na primeira linha?

Vamos renomear as colunas do DataFrame:

In [ ]:
df.columns = ['CodLoja', 'TipoLoja', 'Cidade', 'CustoOperacional','NumFunc', 'FatDia',
              'NumClientesDia', 'CustoAquiPorCli', 'LojaPropria', 'VendeOnline', 'Perfil', 'LojaCentro']
df.head(3)

e agora vamos remover a primeira linha

In [ ]:
df.drop(0, inplace=True)

df.head(3)

Agora, vamos conferir os tipos de dados das colunas.

In [ ]:
df.dtypes

Vamos alterar as colunas numéricas:

In [ ]:
df

In [ ]:
df['CustoOperacional'] = df['CustoOperacional'].str.replace(',','').astype(float)
df['NumFunc'] = df['NumFunc'].astype(int)
df['FatDia'] = df['FatDia'].str.replace(',','').astype(float)
df['NumClientesDia'] = df['NumClientesDia'].astype(float)
df['CustoAquiPorCli'] = df['CustoAquiPorCli'].astype(float)

In [ ]:
df.dtypes

Agora, vamos definir as variáveis categóricas:

In [ ]:
df['TipoLoja'] = df['TipoLoja'].astype('category')
df['Cidade'] = df['Cidade'].astype('category')

df['LojaPropria'] = df['LojaPropria'].astype('category')
df['VendeOnline'] = df['VendeOnline'].astype('category')
df['Perfil'] = df['Perfil'].astype('category')
df['LojaCentro'] = df['LojaCentro'].astype('category')

Assim, os tipos ficam:

In [ ]:
df.dtypes

O que temos até agora?!

In [ ]:
df.head(2)

Quantas linhas (registros) e colunas?

In [ ]:
df.shape

### Filtrando dados
Vamos aprender como selecionar apenas parte dos registros?

Vamos selecionar apenas as lojas da cidade de São Paulo:

In [ ]:
df.loc[df["Cidade"] == "São Paulo"]

Agora, selecione apenas as lojas de Acessórios:

In [ ]:
df.loc[df['TipoLoja'] == 'Acessórios']

Vamos trazer apenas as colunas TipoLoja e FatDia da cidade de São Paulo:

In [ ]:
filtro_linhas = df['Cidade'] == 'São Paulo'
df.loc[filtro_linhas, ['TipoLoja', 'FatDia']]

Agora, apenas as lojas com faturamento diário acima de 12.000,00:

In [ ]:
df.loc[df['FatDia'] > 12_000]

E as lojas de Acessórios que vendem até  3000?

**Atenção!** Se está filtrando por duas colunas, sempre utilize parênteses para envolver os critérios `( ) & ( )`

In [ ]:
df.loc[(df['FatDia'] <= 3_000) & (df['TipoLoja'] == 'Acessórios')]

E somente as colunas Cidade e CustoOperacional, mas mantendo todas as linhas?

In [ ]:
df.loc[:, ['Cidade', 'CustoOperacional']]

## Análise exploratória

Agora, vamos utilizar alguns recursos de estatística e bibliotecas de visualização para conhecermos um pouco melhor os nossos dados.

### Verificando *missing values*

É comum que as bases tenham valores faltantes. Vamos conferir se alguma coluna possui valores faltantes:

In [ ]:
df

Existe toda uma literatura e técnicas que podem ser utilizadas em *missing imputation*.

Primeiro, vamos conferir se eles existem:

In [ ]:
df.isna().sum()

Estranho, não?

Consegue identificar por que isto acontece?

R:

Vamos substituir as strings vazias por NaN (not a number)

In [ ]:
df.replace('', np.nan, inplace=True)

In [ ]:
df.dtypes

E contar os missings novamente:

In [ ]:
df.isna().sum()

Vamos utilizar a biblioteca *missingno* para produzir algo mais visual?!

In [ ]:
mn.matrix(df)

Quais colunas possuem missings?

R: "VendeOnline" e "LojaCentro"

Como podemos corrigir?!

R: Substituindo os valores faltantes por `"0"` em ambos os casos

Vamos remover de nossa análise a última coluna:

In [ ]:
df.drop("LojaCentro", axis=1, inplace=True)
df.head()

E arbitrariamente preencher os missings em VendeOnline com 1:

In [ ]:
df['VendeOnline'].fillna("1", inplace=True)

Conferindo os missings novamente:

In [ ]:
mn.matrix(df)

### Ordenando os dados

Como conseguimos ordenar os dados?

In [ ]:
df.sort_values(by='Cidade')

Será que conseguimos ordenar pelo faturamento em ordem crescente?

In [ ]:
df.sort_values(by='FatDia', ascending=True)

Será que conseguimos ordenar pelo faturamento em ordem decrescente?

In [ ]:
df.sort_values(by='FatDia', ascending=False)

### Criando tabelas de frequência

Conseguimos contar as ocorrências? Por exemplo, e se quiséssemos saber quantas lojas existem em cada cidade?

In [ ]:
df['Cidade'].value_counts()

E se quisessemos saber a porcentagem de lojas em cada cidade?

In [ ]:
df['Cidade'].value_counts(normalize=True)

**Exercício 1** Qual a porcentagem de lojas que são de Eletrônicos?

In [ ]:
TipoLoja_counts = df["TipoLoja"].value_counts(normalize=True)
TipoLoja_counts_eletronicos = TipoLoja_counts["Eletrônicos"]

f"{TipoLoja_counts_eletronicos*100:.2f}% das lojas são de Eletrônicos."

**Exercício 2** Considerando apenas São Paulo, qual a porcentagem de lojas que são de Eletrônicos?

In [ ]:
TipoLoja_counts_sao_paulo = df.loc[df["Cidade"] == "São Paulo", "TipoLoja"].value_counts(normalize=True)
TipoLoja_counts_sao_paulo_eletronicos = TipoLoja_counts_sao_paulo["Eletrônicos"]

f"{TipoLoja_counts_sao_paulo_eletronicos*100:.2f}% das lojas de São Paulo são de Eletrônicos."

Podemos exibir a tabela de frequências de forma gráfica?

In [ ]:
cont = df['Cidade'].value_counts(normalize=True)
cont.plot.bar();

In [ ]:
cont

In [ ]:
cont.reset_index()

In [ ]:
dados = cont.reset_index()

px.bar(dados, x='index', y='Cidade',
       color='index', labels={'index': 'Cidade'},
       title='Freq. Loja por Cidade')

### Medidas Resumo

E se quiséssemos saber a média de faturamento por dia?

In [ ]:
df['FatDia'].mean()

E os valores máximo, mínimo e mediana?!

In [ ]:
df['FatDia'].max()

In [ ]:
df['FatDia'].min()

In [ ]:
df['FatDia'].median()

Podemos resumir estas estatísticas em um único comando:

In [ ]:
df.describe()

### Distribuição

Faz sentido realizar uma contagem na variável de Faturamento?

Podemos analisar a distribuição.

In [ ]:
df['FatDia'].plot.hist();

In [ ]:
df['FatDia'].plot.hist(bins=8);

In [ ]:
px.histogram(df['FatDia'],
             labels={'value': 'Faturamento'},
             title = 'Distribuição do faturamento')

In [ ]:
df_hist = df.loc[:, ['TipoLoja', 'FatDia']]

px.histogram(df_hist, x = 'FatDia',
             color='TipoLoja', opacity=0.75)

E como plotar diversos gráficos na mesma célula?

In [ ]:
fig = plt.figure(figsize=(10, 6))

plt.subplot(2, 2, 1)
df['FatDia'].plot.hist()
plt.title('Faturamento Dia')

plt.subplot(2, 2, 2)
df['CustoOperacional'].plot.hist()
plt.title('Custo Operacional')

plt.subplot(2, 2, 3)
df['NumClientesDia'].plot.hist()
plt.title('Média Cliente por Dia')

plt.subplot(2, 2, 4)
df['CustoAquiPorCli'].plot.hist()
plt.title('Custo médio Aquisição de Clientes')

plt.tight_layout()
plt.show()

### Agrupamentos

Como podemos descobrir o Faturamento médio?

In [ ]:
df['FatDia'].mean()

E para descobrir o faturamento médio por Tipo de Loja?

In [ ]:
group_df =  df.groupby(by='TipoLoja')['FatDia'].mean()
group_df

In [ ]:
fig = plt.figure(figsize=(8, 4))
group_df.plot.bar()
plt.title('Faturamento médio porb Tipo de Loja')
plt.show()

**Exercício 3** Descubra o custo médio de aquisição de cliente para lojas que vendem e não vendem online.

In [ ]:
df.groupby("VendeOnline").mean(numeric_only=True)["CustoAquiPorCli"]

**Exercício 4** Descubra o faturamento médio para lojas que são e não são próprias.

In [ ]:
df.loc[df["LojaPropria"] == "0", "FatDia"].mean()

### Tabelas cruzadas

E se quiséssemos realizar contagens de forma a considerar duas variáveis. Por exemplo, contar por Venda Online e Tipo de Loja?

Neste caso, temos uma situação de análise **bivariada** de variáveis qualitativas:

In [ ]:
ct = pd.crosstab(df['TipoLoja'], df['VendeOnline'])
ct

Podemos utilizar o **normalize** para normalizar para frequência relativa nas linhas, colunas ou toda a tabela cruzada.

Veja mais na documentação https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html

In [ ]:
ct = pd.crosstab(df['TipoLoja'], df['VendeOnline'], normalize='columns')
ct

Estes zeros e uns ficam estranhos, não?! Vamos renomear as categorias!

In [ ]:
df['VendeOnline'] = df['VendeOnline'].cat.rename_categories({'0': 'Não', '1': 'Sim'})
df['LojaPropria'] = df['LojaPropria'].cat.rename_categories({'0': 'Não', '1': 'Sim'})
df.head()

Agora, o crosstab fica melhor:

In [ ]:
# o parâmetro normalize pode ser index, columns ou all
ct = pd.crosstab(df['TipoLoja'], df['VendeOnline'], normalize='columns')
ct

**Exercício 5** Crie:

a) uma alternativa para analisar como os tipos de lojas se distribuem pelas cidades.

In [ ]:
pd.crosstab(df['TipoLoja'], df['Cidade'], normalize=1)

b) Dentre as lojas de Eletrônicos, qual a porcentagem das lojas que está localizada no Rio de Janeiro?

R: 8,6957%

c) Dentre as lojas de Belo Horizonte, qual a porcentagem das lojas é do tipo Acessórios?

R: 42,8571%

### Correlação entre variáveis

Podemos analisar a correlação entre as variáveis utilizando uma funcionalide da biblioteca pandas:

In [ ]:
df.corr()

ou pelo uso da biblioteca seaborn:

In [ ]:
sb.heatmap(df.corr(numeric_only=True));

Ou pelo plotly express:

In [ ]:
px.imshow(df.corr(numeric_only=True))

### Pivot Table

E se for necessário descobrir o faturamento médio de cada tipo de loja em cada cidade?

In [ ]:
pivot = pd.pivot_table(df, index='TipoLoja', columns='Cidade',
               values=['FatDia'], aggfunc=np.mean)

pivot

In [ ]:
fig = plt.figure(figsize=(18, 5))
pivot.plot.bar()
plt.title('Faturameto médio por categoria e cidade')
plt.show()

In [ ]:
pivot_aux = pivot.unstack().reset_index()
pivot_aux

In [ ]:
px.bar(pivot_aux, x='Cidade', y=0, color='TipoLoja', barmode='group')

In [ ]:
px.bar(pivot_aux, x='Cidade', y=0, color='TipoLoja', barmode='stack')

### Criando colunas novas

Como podemos fazer para entender o ticket médio? Temos a média de clientes por dia e o faturamento por dia. Como obter o ticket médio?

In [ ]:
df.head()

In [ ]:
df['TicketMedio'] = df['FatDia'] / df['NumClientesDia']
df.head()

E para criar categorias de custo? Abaixo de cinco mil é custo BAIXO, de cinco até oito é MÉDIO, acima de oito mil é ALTO?

In [ ]:
def cat_custo(valor):
    if valor < 5000:
        return 'BAIXO'
    elif valor < 8000:
        return 'MEDIO'
    else:
        return 'ALTO'

In [ ]:
df['CatCusto'] = df['CustoOperacional'].apply(cat_custo)
df.head()

**Exercício 6** Conte quantas lojas estão em cada categoria de custo.

In [ ]:
df["CatCusto"].value_counts()

**Exercício 7** Calcule o faturamento médio por Categoria de custo.

In [ ]:
df.groupby("CatCusto").mean(numeric_only=True)["FatDia"]

**Exercício 8** Cruze a categoria de custo com o tipo de loja para contar quantas estão em cada categoria conjunta.

In [ ]:
pd.crosstab(df["CatCusto"], df["TipoLoja"])

**Exercício 9** Cruze a categoria de custo com o tipo de loja, para descobrir o faturamento médio.

In [ ]:
pd.crosstab(df["CatCusto"], df["TipoLoja"], values=df["FatDia"], aggfunc="mean")

**Exercício 10** Sugira categorias para o número de funcionários e clientes. Analise o faturamento médio.

In [ ]:
cross_func_clientes_faturamento_media = pd.crosstab(df["NumFunc"], df["NumClientesDia"], values=df["FatDia"], aggfunc="mean")

sb.heatmap(cross_func_clientes_faturamento_media)

In [ ]:
df["Rendimento"] = df["FatDia"] / (df["NumFunc"] * df["NumClientesDia"])

sb.histplot(df["Rendimento"])

In [ ]:
slice_rendimento_min = 5
slice_rendimento_max = 200
slice_rendimento = df.loc[(df["Rendimento"] > 7) & (df["Rendimento"] < 200), "Rendimento"]

sb.histplot(slice_rendimento, bins=30)

In [ ]:
describe_rendimento = slice_rendimento.describe()
describe_rendimento

In [ ]:
def cat_rendimento(valor):
    if valor < describe_rendimento["mean"] - describe_rendimento["std"]:
        return 'BAIXO'
    elif valor < describe_rendimento["mean"] + describe_rendimento["std"]:
        return 'MEDIO'
    else:
        return 'ALTO'

df['CatRendimento'] = df['Rendimento'].apply(cat_rendimento).astype("category")
df.head()

**Exercício 11** Crie uma coluna de região geográfica.

# Exportar o DataFrame para arquivos

Será que conseguimos exportar o pandas DataFrame para algum arquivo?

Conseguimos sim!

Para exportar para Excel:

In [ ]:
df.to_excel('lojas.xlsx')

e para CSV:

In [ ]:
df.to_csv('lojas.csv')

# Extração de textos com PyPDF2

Caso o seu PDF contenha textos ao invés de tabelas, podemos utilizar o PyPDF2.

Vamos importar a biblioteca:

In [ ]:
import PyPDF2 as pp
import urllib.request

Caso o import não seja realizado, descomente a próxima linha e faça a instalação

In [ ]:
# !pip install PyPDF2

Vamos fazer o download do arquivo PDF

In [ ]:
pdf_url = 'https://atd-insper.s3.us-east-2.amazonaws.com/aula02/ppc_computacao.pdf'

response = urllib.request.urlopen(pdf_url)

arq = open('ppc_computacao.pdf', 'wb')
arq.write(response.read())
arq.close()

e abrir o Arquivo

In [ ]:
pp_reader = pp.PdfFileReader(open('ppc_computacao.pdf', 'rb')) 

Podemos buscar informações do documento, como o número de páginas

In [ ]:
pp_reader.documentInfo

In [ ]:
print(f'O arquivo tem {pp_reader.numPages} páginas')

Ou ler uma página específica do PDF

In [ ]:
texto = pp_reader.getPage(0).extractText()
print(texto)

Vamos ler as 10 primeiras páginas, armazenando cada página em uma lista

In [ ]:
lista_pg = []
for i in range(10):
    texto = pp_reader.getPage(i).extractText()
    lista_pg.append(texto)

Para uma melhor visualização, vamos podemos remover as quebras de linhas com **replace**

In [ ]:
print(lista_pg[7].replace('\n', ''))

In [ ]:
lista_pg[7].count('sistemas')

**Exercício 12** Leia todas as páginas do PDF `ppc_computacao.pdf` e armazene o conteúdo em uma lista.

In [ ]:
pages = [
    pp_reader.getPage(index).extractText()
    for index in range(pp_reader.numPages)
]

**Exercício 13** Para cada página, faça algumas limpezas:
- Remova stop-words
- Remova quebras de linhas
- Remova pontuações

In [ ]:
punctuation = [',', '.', '?', '!']


def remove_words(text, words):
    return " ".join([word for word in text.split() if word not in words])

def strip(text):
    return text.strip().replace("\n", "")

def remove_portuguese_punctuation(text):
    for character in punctuation:
        text = text.replace(character, "")

    return text

def remove_portuguese_stop_words(text):
    return remove_words(text, [
        "de","a","o","que","e","do","da","em","um","para","é","com","não","uma","os","no","se","na","por","mais","as","dos","como","mas","foi","ao","ele","das","tem","à","seu","sua","ou","ser","quando","muito","há","nos","já","está","eu","também","só","pelo","pela","até","isso","ela","entre","era","depois","sem","mesmo","aos","ter","seus","quem","nas","me","esse","eles","estão","você","tinha","foram","essa","num","nem","suas","meu","às","minha","têm","numa","pelos","elas","havia","seja","qual","será","nós","tenho","lhe","deles","essas","esses","pelas","este","fosse","dele","tu","te","vocês","vos","lhes","meus","minhas","teu","tua","teus","tuas","nosso","nossa","nossos","nossas","dela","delas","esta","estes","estas","aquele","aquela","aqueles","aquelas","isto","aquilo","estou","está","estamos","estão","estive","esteve","estivemos","estiveram","estava","estávamos","estavam","estivera","estivéramos","esteja","estejamos","estejam","estivesse","estivéssemos","estivessem","estiver","estivermos","estiverem","hei","há","havemos","hão","houve","houvemos","houveram","houvera","houvéramos","haja","hajamos","hajam","houvesse","houvéssemos","houvessem","houver","houvermos","houverem","houverei","houverá","houveremos","houverão","houveria","houveríamos","houveriam","sou","somos","são","era","éramos","eram","fui","foi","fomos","foram","fora","fôramos","seja","sejamos","sejam","fosse","fôssemos","fossem","for","formos","forem","serei","será","seremos","serão","seria","seríamos","seriam","tenho","tem","temos","tém","tinha","tínhamos","tinham","tive","teve","tivemos","tiveram","tivera","tivéramos","tenha","tenhamos","tenham","tivesse","tivéssemos","tivessem","tiver","tivermos","tiverem","terei","terá","teremos","terão","teria","teríamos","teriam"
    ])

def normalize_text(text):
    return remove_portuguese_stop_words(remove_portuguese_punctuation(strip(text.lower())))

normalized_pages = list(map(normalize_text, pages))

**Exercício 14** Crie uma função que retorne todas as páginas do PDF `ppc_computacao.pdf` que contém uma determinada palavra passada como parâmetro

In [ ]:
def find(word):
    return [
        index
        for index, page in enumerate(pages)
        if word.lower() in page
    ]

find("Insper")

**Exercício 15** Crie uma função que retorne todas as páginas onde uma palavra ou termo ocorre, além disso, deve retornar a quantidade.

Dica: utilizar uma lista com tuplas ou dicionário.

In [ ]:
def word_counts(word):
    return {
        index: page.count(word.lower())
        for index, page in enumerate(pages)
        if word.lower() in page
    }

word_counts("Computação")

# Alternativas:

Algumas outras bibliotecas que podem ser utilizadas para extração de textos de PDFs:

- tabula-py
- tika
- xpdf-python

# Links interessantes da documentação do Camelot:
- https://camelot-py.readthedocs.io/en/master/user/how-it-works.html#lattice
- https://camelot-py.readthedocs.io/en/master/user/quickstart.html#read-the-pdf
- https://camelot-py.readthedocs.io/en/master/user/advanced.html#detect-short-lines
- https://camelot-py.readthedocs.io/en/master/user/faq.html#does-camelot-work-with-image-based-pdfs